In [17]:
import pandas as pd
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier,_tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.tree import export_graphviz
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [18]:
# reading the csv file and assigning x,y,y1 for from columns
training = pd.read_csv('chatbot_Training.csv')
testing  = pd.read_csv('chatbot_Testing.csv')
cols     = training.columns
cols     = cols[:-1]
x        = training[cols]
y        = training['prognosis']
y1       = y
#grouping the data of same criteria
reduced_data=training.groupby(training['prognosis']).max()

In [19]:
# fitting the model to string to integers ie., converting words to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

In [20]:
# training and testing of data using train_test_split from sklearn.model selection
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx    = testing[cols]
testy    = testing['prognosis']  
testy    = le.transform(testy)

In [21]:
#appling desicion tree algo and printing result
from sklearn.model_selection import cross_val_score
clf1  = DecisionTreeClassifier()
clf = clf1.fit(x_train,y_train)
print(clf.score(x_train,y_train))
print ("cross result========")
scores = cross_val_score(clf, x_test, y_test, cv=3)
print (scores)
print (scores.mean())
print(clf.score(testx,testy))

1.0
cross result========
[0.97232472 0.97966728 0.97227357]
0.9747551911748323
1.0


In [22]:
#to check the permutation of data for impurities
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols


for f in range(10):
    print("%d. feature %d - %s (%f)" % (f + 1, indices[f], features[indices[f]] ,importances[indices[f]]))

1. feature 35 - loss_of_appetite (0.045968)
2. feature 55 - congestion (0.026703)
3. feature 93 - internal_itching (0.026649)
4. feature 79 - hip_joint_pain (0.026626)
5. feature 104 - increased_appetite (0.026390)
6. feature 126 - silver_like_dusting (0.025712)
7. feature 73 - swollen_extremeties (0.025479)
8. feature 118 - blood_in_sputum (0.024874)
9. feature 131 - yellow_crust_ooze (0.024798)
10. feature 2 - nodal_skin_eruptions (0.024735)


In [23]:
print("Please reply Yes or No for the following symptoms")
def print_disease(node):
    #print(node)
    node = node[0]
    #print(len(node))
    val  = node.nonzero() 
    #print(val)
    disease = le.inverse_transform(val[0])
    return disease
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    #print(tree_)
    feature_name = [feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
                    for i in tree_.feature]
    #print("def tree({}):".format(", ".join(feature_names)))
    symptoms_present = []
    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print(name + " ?")
            ans = input()
            ans = ans.lower()
            if ans == 'yes':
                val = 1
            else:
                val = 0
            if  val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            print( "You may have " +  present_disease )
            red_cols = reduced_data.columns 
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
            print("symptoms present  " + str(list(symptoms_present)))
            print("symptoms given "  +  str(list(symptoms_given)) )  
            confidence_level = (1.0*len(symptoms_present))/len(symptoms_given)
            print("confidence level is " + str(confidence_level))

    recurse(0, 1)

tree_to_code(clf,cols)

Please reply Yes or No for the following symptoms
congestion ?
0
internal_itching ?
0
hip_joint_pain ?
0
increased_appetite ?
0
silver_like_dusting ?
0
swollen_extremeties ?
0
blood_in_sputum ?
0
yellow_crust_ooze ?
0
nodal_skin_eruptions ?
0
unsteadiness ?
0
weight_loss ?
0
prominent_veins_on_calf ?
0
fluid_overload.1 ?
0
depression ?
0
lack_of_concentration ?
0
mild_fever ?
0
muscle_pain ?
0
neck_pain ?
0
altered_sensorium ?
0
slurred_speech ?
0
movement_stiffness ?
0
shivering ?
0
belly_pain ?
1
continuous_feel_of_urine ?
0
rusty_sputum ?
0
receiving_unsterile_injections ?
0
itching ?
0
patches_in_throat ?
0
family_history ?
0
stomach_pain ?
0
coma ?
0
dark_urine ?
0
pus_filled_pimples ?
0
bloody_stool ?
0
sunken_eyes ?
0
breathlessness ?

watering_from_eyes ?

dehydration ?
0
scurring ?
0
sweating ?
0
pain_behind_the_eyes ?
0
joint_pain ?
0
weakness_of_one_body_side ?
0
extra_marital_contacts ?
0
chest_pain ?
0
diarrhoea ?
1
foul_smell_of urine ?
0
high_fever ?
1
spinning_movements